In [1]:
from lxml.html import fromstring as to_tree
import csv
import requests

In [67]:
with open('energy.html', 'rb') as f:
    tree = to_tree(f.read())

In [2]:
def request(url):
    response = requests.get(url)

    if response.status_code != 200:
        log.append(url)
        raise Exception(f'ERROR CODE {response.status_code}')
        
    return response

In [7]:
def parse_article(url):
    req = request(url)
    tree = to_tree(req.text)
    title = ' '.join(tree.xpath('//*[@class="news-header"]/text()')).strip()
    content = '\n'.join(tree.xpath('//*[@class="purified_text"]/p/text()'))
    return url, title, content

In [12]:
from tqdm.notebook import trange, tqdm

In [18]:
base_url = 'https://energybase.ru'
n_page = 'https://energybase.ru/news/electric-power?NewsSearch%5Bpublished_at%5D=allTime&newsPage={}&per-page=10#news-list'
contents = []
log = []

for page in trange(90):
    n_page_req = request(n_page.format(page))
    n_page_tree = to_tree(n_page_req.text)
    urls = n_page_tree.xpath('//a[@class= "news-item__header-link"]/@href')
    for rel_url in urls:
        url = base_url + rel_url
        contents.append(parse_article(url))

In [19]:
import pandas as pd
df = pd.DataFrame(contents)

In [21]:
df.columns = ['url', 'title', 'content']
df.to_csv('elec_base.csv', index=False)